In [ ]:
# Genetic Algorithm for Allocating a Progressive Carbon Dividend
                                    
# please note that NVIDIA RAPIDS libraries cannot be used without an NVIDIA GPU.

In [ ]:
# Huge thanks to Ahmed F. Gad for creating "pygad", an open source python library that provided the framework to set up my genetic algorithm

# Pygad Cite:

# @article{gad2023pygad,
#   title={Pygad: An intuitive genetic algorithm python library},
#   author={Gad, Ahmed Fawzy},
#   journal={Multimedia Tools and Applications},
#   pages={1--14},
#   year={2023},
#   publisher={Springer}
# }

In [ ]:
import math
import random
import numpy as np
import pandas as pd    
from dask_ml.model_selection import train_test_split
import pygad
import xgboost as xgb 
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
from dask_ml.ensemble import BlockwiseVotingRegressor
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_predict
import time
import cuml
import cudf
import dask
import dask_cudf as dc
dask.config.set({"dataframe.backend": "cudf"})
dask.config.set({'large-graph-warning-threshold': False})
from cuml.ensemble import RandomForestRegressor as cuRF
from dask_ml.model_selection import GridSearchCV 
import dask_cudf
import dask.array as da
import dask.dataframe as dd
import dask.distributed
from dask.distributed import Client, LocalCluster
import cupy as cp
from cuml import ForestInference
import joblib
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
!nvidia-smi

In [ ]:
import os
import dask_cuda

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(os.environ['CUDA_VISIBLE_DEVICES'])
cluster = dask_cuda.LocalCUDACluster(n_workers=1)
client = Client(cluster)
client

In [ ]:
cp.random.seed(seed=10)

In [ ]:
# Survey data cleaning and analysis

# SHED 2022, https://www.federalreserve.gov/consumerscommunities/shed_data.htm
df = pd.read_excel(r"SHED_2022_Cleaned.xlsx", index_col='CaseID')

# See the data cleaning file

In [ ]:
# Creating the model

In [ ]:
# Indicators

In [ ]:
def indicator_distance_from_poverty(df):
    poverty_line_2024 = 15060  # Poverty line for 1 person household (2024)
    # https://aspe.hhs.gov/topics/poverty-economic-mobility/poverty-guidelines
    distance_from_poverty = (df["per_person_income"] / poverty_line_2024) * 100
    return distance_from_poverty
                
percentages = indicator_distance_from_poverty(df)
percentages = percentages.round(3)
df["distance_from_poverty"] = percentages

In [ ]:
def indicator_distance_from_mean_income(df):
    mean_income = da.mean(df["per_person_income"]).compute()
    distance_from_mean_income = (df["per_person_income"] / mean_income) * 100
    return distance_from_mean_income

percentages = indicator_distance_from_mean_income(df)
percentages = percentages.round(3)
df["distance_from_mean_income"] = percentages

In [ ]:
# Classification indicator
df['FSRI Score'].describe()
df["target"] = np.select(
    [df["FSRI Score"] >= 0.77,
     (df["FSRI Score"] >= 0.64) & (df["FSRI Score"] <= 0.77),
     (df["FSRI Score"] >= 0.476250) & (df["FSRI Score"] <= 0.64),
     df["FSRI Score"] <= 0.476250],
    ["Extremely Stable", "Somewhat Stable", "Somewhat Unstable", "Extremely Unstable"]
)

categories = ["Extremely Unstable","Somewhat Unstable","Somewhat Stable","Extremely Stable"]
oe = OrdinalEncoder(categories = [categories])
df["target"] = oe.fit_transform(df[["target"]])
df["target"]
# 0: Extremely Unstable
# 1: Somewhat Unstable
# 2: Somewhat Stable
# 3: Extremely Stable

X_class = df[["D1A","3_month_expense_coverage","BK1","I9","I20","I21_a","affect_of_price_inc","EF3_h","EF5A","EF7","E2B","ppeducat","ppemploy","ppfs1482","atleast_okay"]]
y_class = df["target"]

categorical_features = ["D1A","3_month_expense_coverage","BK1","I9","I20","I21_a","affect_of_price_inc","EF3_h","EF5A","EF7","E2B","ppeducat","ppemploy","ppfs1482","atleast_okay"]
one_hot = OneHotEncoder()
transformer = ColumnTransformer([("one_hot", one_hot, categorical_features)], remainder="passthrough")
X_class_transformed = transformer.fit_transform(X_class)

class_df = pd.DataFrame(X_class_transformed)
caseid_index = df.index
class_df.index = caseid_index

# XGBoost model
xgb_class = xgb.XGBClassifier(random_state=10, n_estimators=1000, gamma=0.1, max_depth=3, learning_rate=0.1)
probabilities = cross_val_predict(xgb_class, class_df, y_class, cv=5, method='predict_proba')
predictions = cross_val_predict(xgb_class, class_df, y_class, cv=5)
f1 = f1_score(y_class, predictions, average='micro')
print("F1 Score: ", f1)

In [ ]:
len(probabilities)

In [ ]:
# Sum of probabilities
weights = [1, 2, 4, 8]
weights = np.array(weights)
weighted_proba = probabilities * weights
weighted_sum = np.sum(weighted_proba, axis=1)

df["weighted_sum"] = weighted_sum

In [ ]:
indicator_df = df[["distance_from_poverty", "distance_from_mean_income", "weighted_sum"]]
indicator_df = cudf.DataFrame.from_pandas(indicator_df)
indicator_df = dask_cudf.from_cudf(indicator_df, npartitions=4)

In [ ]:
pdf = df

In [ ]:
df = cudf.DataFrame.from_pandas(df)
df = dask_cudf.from_cudf(df, npartitions=4)

In [ ]:
df.head()

In [ ]:
pdf.drop(columns=pdf.columns, inplace=True)
pdf.head()

In [ ]:
print(type(indicator_df))
print(type(df))

In [ ]:
indicator_df.head()

In [ ]:
# Regression to calculate weighted sum based on income
X_sum = df[["per_person_income"]].astype(cp.float32)
y_sum = indicator_df["weighted_sum"].astype(cp.float32)
X_sum_train, X_sum_test, y_sum_train, y_sum_test = train_test_split(X_sum, y_sum, test_size=0.2, random_state=10, shuffle=True)

In [ ]:
%%time
rfr_sum = cuRF(random_state=10, max_depth = 20, n_estimators=1000, min_samples_split=15, n_streams=1)
rfr_sum.fit(X_sum_train, y_sum_train)
fm_sum = rfr_sum.convert_to_fil_model(output_class=False)

In [ ]:
def predict_indicators(income):
    #Indicators
    poverty_line_2024 = 15060
    distance_poverty_percent = (income / poverty_line_2024) * 100
    
    mean_income = 60168.54
    distance_from_mean_income = (income / mean_income) * 100
    
    income_da = da.array([[income]])
    income_da = income_da.compute()
    weighted_sum = fm_sum.predict(income_da)
    
    return [distance_poverty_percent, distance_from_mean_income, weighted_sum[0]]

In [ ]:
%%time 
income = 10000
test = predict_indicators(income)
print(test)

In [ ]:
indicator_df.head()

In [ ]:
# Model

In [ ]:
X_indicator = indicator_df[["distance_from_poverty", "distance_from_mean_income", "weighted_sum"]].astype(cp.float32)
y = df['FSRI Score'].astype(cp.float32)
X_indicator_train, X_indicator_test, y_train, y_test = train_test_split(X_indicator, y, test_size=0.2, random_state=10)

In [ ]:
# RFR model
rfr_indicator = cuRF(random_state=10, max_depth = 20, n_estimators=1000, min_samples_split=15, n_streams=1)
rfr_indicator.fit(X_indicator_train, y_train)
fm_indicator = rfr_indicator.convert_to_fil_model(output_class=False)

In [ ]:
X_indicator_test_da = X_indicator_test.compute()
X_indicator_test_cp = X_indicator_test_da.to_cupy()
y_pred_rfr_indicator = fm_indicator.predict(X_indicator_test_cp)

y_test_da = y_test.compute()
y_test_np = y_test_da.to_numpy()
y_test_cp = y_test_da.to_numpy()
y_pred_rfr_indicator_np = y_pred_rfr_indicator.get()

r2_rfr_indicator = r2_score(y_test_np, y_pred_rfr_indicator_np)
mae_rfr_indicator = mean_absolute_error(y_test_np, y_pred_rfr_indicator_np)
mse_rfr_indicator = mean_squared_error(y_test_np, y_pred_rfr_indicator_np)

print(f"R2 (RF): {r2_rfr_indicator}")
print(f"Mean Absolute Error (RF): {mae_rfr_indicator}")
print(f"Mean Squared Error (RF): {mse_rfr_indicator}")

In [ ]:
# Ridge Model
from cuml import Ridge
rge_indicator = Ridge(alpha=100, output_type="cupy", solver="svd", fit_intercept=True)
rge_indicator.fit(X_indicator_train, y_train)

y_pred_rge_indicator = rge_indicator.predict(X_indicator_test_cp)
y_pred_rge_indicator_np = y_pred_rge_indicator.get()

r2_rge_indicator = r2_score(y_test_np, y_pred_rge_indicator_np)
mae_rge_indicator = mean_absolute_error(y_test_np, y_pred_rge_indicator_np)
mse_rge_indicator = mean_squared_error(y_test_np, y_pred_rge_indicator_np)

print(f"R2 (Ridge): {r2_rge_indicator}")
print(f"Mean Absolute Error (Ridge): {mae_rge_indicator}")
print(f"Mean Squared Error (Ridge): {mse_rge_indicator}")

In [ ]:
combined_preds = (y_pred_rfr_indicator_np * 0.75) + (y_pred_rge_indicator_np * 0.25)
r2_score_combined = r2_score(y_test_np, combined_preds)
mae_score_combined = mean_absolute_error(y_test_np, combined_preds)
mse_score_combined = mean_squared_error(y_test_np, combined_preds)

print(f"R2 (combined): {r2_score_combined}")
print(f"Mean Absolute Error (combined): {mae_score_combined}")
print(f"Mean Squared Error (combined): {mse_score_combined}")

In [ ]:
income = df["per_person_income"] 
per_person_inc = da.array(income)
per_person_inc = per_person_inc.compute()
type(per_person_inc)

In [ ]:
def predict_indicators(income):
    # Indicators
    poverty_line_2024 = 15060
    distance_poverty_percent = (income / poverty_line_2024) * 100
    
    mean_income = 60168.54
    distance_from_mean_income = (income / mean_income) * 100
    
    weighted_sum = fm_sum.predict(cp.array([[income]], dtype=cp.float32))
    return [distance_poverty_percent, distance_from_mean_income, weighted_sum[0]]

In [ ]:
%%time
def final_score_calc(per_person_inc):
    final_preds = []
    for inc in per_person_inc:
        indicators = predict_indicators(inc)
        indicator_cp = cp.array([indicators], dtype=cp.float32)
        final_pred1 = fm_indicator.predict(indicator_cp)
        final_pred2 = rge_indicator.predict(indicator_cp)
        final_pred = (final_pred1 * 0.75) + (final_pred2 * 0.25)
        final_preds.append(final_pred)
    
    return final_preds

income = df["per_person_income"] 
per_person_inc = da.array(income)
per_person_inc = per_person_inc.compute()
original_FSRI = final_score_calc(per_person_inc)
len(original_FSRI)

In [ ]:
original_FSRI_cp = cp.array(original_FSRI)
original_FSRI_np = original_FSRI_cp.get()
pdf["original_FSRI"] = 0
pdf["original_FSRI"] = original_FSRI_np
pdf.head()

In [ ]:
# THE GENETIC ALGORITHM

In [ ]:
# Setup for fitness func

gene_space = [20592000000.0, 21621600000.0, 22651200000.0, 23680800000.0, 24710400000.0, 25740000000.0, 26769600000.0, 27799200000.0, 28828800000.0, 29858400000.0, 30888000000]
# Setting the ranges of the min and max that each decile can receive, (no more/less than 20% of an equal per capita dividend distribution scenario)

total_rev = 257400000000 #Total revenue generated from the carbon tax, CBO estimates

amount_of_ppl_in_decile = 34200000

# Based off of updated burden calculations (repurposed from Fremstad & Paul 2019)
    
# The following were generated from Anders Fremstad's "The Impact of a Carbon Tax on Inequality".
# I edited & updated these programs to fit my specific scenario and regenerated the results. These regenerated results served as some of the inputs into my genetic algorithm.

increased_costs_actual = [488.53, 582.45, 722.86, 776.35, 843.54, 941.06, 1027.09, 1144.42, 1261.38, 1528.00] # Estimated costs per decile (in ascending decile order). Average amount that each decile is expected to have to pay in increased prices in 2024 dollars.
# By individual

In [ ]:
income = df["per_person_income"] 
original_income = income.compute()
original_income = original_income.to_pandas()

decile = df["decile"]
decile = decile.compute()
decile = decile.to_pandas()

In [ ]:
# Fitness value for equal dividends

def fitness_func_benchmark(test_solution):

    # Objectives 1, 2, & 3 - Maximizing FSRI Score

    def dividend_income():
        new_fsri = []
        for inc, dec in zip(original_income, decile):
            dividend_income = test_solution[dec - 1] / amount_of_ppl_in_decile  # Adjusting for zero-indexing
            dividend_net = dividend_income - increased_costs_actual[dec - 1]
            tot_income = inc + dividend_net
            new_fsri.append(tot_income)
        return new_fsri

    total_incomes = dividend_income()

    per_person_inc = cp.array(total_incomes)
    new_FSRI = final_score_calc(per_person_inc)

    new_FSRI_cp = cp.array(new_FSRI)
    new_FSRI_np = new_FSRI_cp.get()
    pdf["new_FSRI"] = new_FSRI_np
    pdf.head()

    pdf["FSRI_diff"] = pdf["new_FSRI"] - pdf["original_FSRI"]
    pdf["decile"] = decile
    
    fitness1 = np.mean(pdf["FSRI_diff"])

    mean_bottom_half = pdf.loc[pdf['decile'] <= 5, "FSRI_diff"].mean()
    fitness2 = mean_bottom_half

    mean_top_half = pdf.loc[pdf['decile'] > 5, "FSRI_diff"].mean()
    fitness3 = mean_top_half
    
    # Penalty for not adding up to total_rev
    # Added because I want the algorithm to gurantee the use of all of the funds to decrease the time until convergence.

    deviation = abs(np.sum(test_solution) - total_rev)
    penalty_rate = 1000
    penalty = deviation * penalty_rate

    fitness1 -= penalty
    fitness2 -= penalty
    fitness3 -= penalty

    return [fitness1, fitness2, fitness3]

test_solution = [25740000000, 25740000000, 25740000000, 25740000000, 25740000000, 25740000000, 25740000000, 25740000000, 25740000000, 25740000000]
benchmark = fitness_func_benchmark(test_solution)
print(f"benchmark score (equal dividends) {benchmark}")

In [ ]:
pdf["decile"] = decile
pdf["per_person_income"] = original_income
pdf["original_FSRI"] = pdf["original_FSRI"]
pdf["equal_new_FSRI"] = pdf["new_FSRI"]
pdf["equal_FSRI_diff"] = pdf["equal_new_FSRI"] - pdf["original_FSRI"]
pdf = pdf.drop(columns = ["new_FSRI", "FSRI_diff"])

In [ ]:
# The fitness func

def fitness_func(ga_instance, solution, solution_idx):

    # Objectives 1, 2, & 3 - Maximizing FSRI Score

    def dividend_income():
        new_fsri = []
        for inc, dec in zip(original_income, decile):
            dividend_income = solution[dec - 1] / amount_of_ppl_in_decile  # Adjusting for zero-indexing
            dividend_net = dividend_income - increased_costs_actual[dec - 1]
            tot_income = inc + dividend_net
            new_fsri.append(tot_income)
        return new_fsri

    total_incomes = dividend_income()

    per_person_inc = cp.array(total_incomes)
    new_FSRI = final_score_calc(per_person_inc)

    new_FSRI_cp = cp.array(new_FSRI)
    new_FSRI_np = new_FSRI_cp.get()
    pdf["new_FSRI"] = 0
    pdf["new_FSRI"] = new_FSRI_np
    pdf.head()

    pdf["FSRI_diff"] = pdf["new_FSRI"] - pdf["original_FSRI"]
    pdf["decile"] = decile
    
    fitness1 = np.mean(pdf["FSRI_diff"])

    mean_bottom_half = pdf.loc[pdf['decile'] <= 5, "FSRI_diff"].mean()
    fitness2 = mean_bottom_half

    mean_top_half = pdf.loc[pdf['decile'] > 5, "FSRI_diff"].mean()
    fitness3 = mean_top_half
    
    # Penalty for not adding up to total_rev
    # Added because I want the algorithm to gurantee the use of all of the funds to decrease the time until convergence.

    deviation = abs(np.sum(solution) - total_rev)
    penalty_rate = 1000
    penalty = deviation * penalty_rate

    fitness1 -= penalty
    fitness2 -= penalty
    fitness3 -= penalty

    return [fitness1, fitness2, fitness3]

In [ ]:
def custom_mutation(offspring, ga_instance):
    for chromosome_idx in range(offspring.shape[0]):
        random_gene_idx = np.random.choice(range(offspring.shape[1]))
        old_value = offspring[chromosome_idx, random_gene_idx]
        new_value = np.random.choice(gene_space)
        difference = new_value - old_value
        offspring[chromosome_idx, random_gene_idx] = new_value
        
        valid_other_gene_indices = None
        if difference >= 0:
            valid_other_gene_indices = [i for i in range(offspring.shape[1]) if i != random_gene_idx and offspring[chromosome_idx, i] - difference >= min(gene_space)]
        else:
            valid_other_gene_indices = [i for i in range(offspring.shape[1]) if i != random_gene_idx and offspring[chromosome_idx, i] - difference <= max(gene_space)]

        if not valid_other_gene_indices:
            offspring[chromosome_idx, random_gene_idx] = old_value
            continue
            
        other_gene_idx = np.random.choice(valid_other_gene_indices)
        new_other_value = offspring[chromosome_idx, other_gene_idx] - difference
        offspring[chromosome_idx, other_gene_idx] = new_other_value
        
    return offspring

In [ ]:
%%time
# Parameters of the GA

initial_gene_value = 25740000000
initial_population = np.full((5, 10), initial_gene_value)

def on_gen(ga_instance):
    print("Generation : ", ga_instance.generations_completed)
    print("Fitness of the best solution :", ga_instance.best_solution()[1])
    print(f"Best Solution: {ga_instance.best_solution()[0]}")

ga_instance = pygad.GA(fitness_func = fitness_func,
                       num_generations = 500,
                       num_parents_mating = 3,
                       sol_per_pop = 5,
                       num_genes = 10,
                       gene_space = gene_space,
                       parent_selection_type = "tournament_nsga2",
                       crossover_type= "scattered",
                       crossover_probability= 0.5,
                       mutation_type=custom_mutation,
                       mutation_probability = 0.10,
                       random_seed = 10,
                       on_generation = on_gen,
                       allow_duplicate_genes=True,
                       initial_population = initial_population
                       )

ga_instance.run()

In [ ]:
ga_instance.plot_fitness()

In [ ]:
best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
print(f"Best Solution: {best_solution}")
print(f"Best Solution Fitness: {best_solution_fitness}")

if ga_instance.best_solution_generation != -1:
    print(f"Best fitness value reached after {ga_instance.best_solution_generation} generations.")

In [ ]:
# Calculate the income and FSRI difference for the new solution

In [ ]:
GA_best_solution = ', '.join(map(str, best_solution))
GA_best_solution = [float(x) for x in GA_best_solution.replace(" ", "").split(",")]

In [ ]:
# GA New FSRI

In [ ]:
def fitness_func_benchmark(test_solution):

    # Objectives 1, 2, & 3 - Maximizing FSRI Score

    def dividend_income():
        new_fsri = []
        for inc, dec in zip(original_income, decile):
            dividend_income = test_solution[dec - 1] / amount_of_ppl_in_decile  # Adjusting for zero-indexing
            dividend_net = dividend_income - increased_costs_actual[dec - 1]
            tot_income = inc + dividend_net
            new_fsri.append(tot_income)
        return new_fsri

    total_incomes = dividend_income()

    per_person_inc = cp.array(total_incomes)
    new_FSRI = final_score_calc(per_person_inc)

    new_FSRI_cp = cp.array(new_FSRI)
    new_FSRI_np = new_FSRI_cp.get()
    pdf["GA_new_FSRI"] = new_FSRI_np
    pdf.head()

    pdf["GA_FSRI_diff"] = pdf["GA_new_FSRI"] - pdf["original_FSRI"]
    pdf["decile"] = decile
    
    fitness1 = np.mean(pdf["GA_FSRI_diff"])

    mean_bottom_half = pdf.loc[pdf['decile'] <= 5, "GA_FSRI_diff"].mean()
    fitness2 = mean_bottom_half

    mean_top_half = pdf.loc[pdf['decile'] > 5, "GA_FSRI_diff"].mean()
    fitness3 = mean_top_half
    
    # Penalty for not adding up to total_rev
    # Added because I want the algorithm to gurantee the use of all of the funds to decrease the time until convergence.

    deviation = abs(np.sum(test_solution) - total_rev)
    penalty_rate = 1000
    penalty = deviation * penalty_rate

    fitness1 -= penalty
    fitness2 -= penalty
    fitness3 -= penalty

    return [fitness1, fitness2, fitness3]

test_solution = GA_best_solution
score = fitness_func_benchmark(test_solution)
print(f"GA solution score: {score}")

In [ ]:
pdf["GA_new_FSRI"] = pdf["GA_new_FSRI"]
pdf["GA_FSRI_diff"] = pdf["GA_new_FSRI"] - pdf["original_FSRI"]

In [ ]:
output_file_path = r"C:\Users\...\Carbon_Dividend_Analysis\Data\Dividend_Data.xlsx"
pdf.to_excel(output_file_path)